# データ確認・前処理

In [21]:
%run 0.2_Function.ipynb

In [22]:
# 2015-2025
df = result_df()

In [23]:
# 年、月、日からdateカラム（yymmdd）を生成
df["レース日"] = (
    df["年"].astype(str).str.zfill(2)
    + df["月"].astype(str).str.zfill(2)
    + df["日"].astype(str).str.zfill(2)
)

# dateカラムをdatetime型に変換
df["レース日"] = pd.to_datetime(df["レース日"], format="%y%m%d", yearfirst=True)

# 不要なカラムを削除
df.drop(
    columns=[
        "馬印",
        "レース印",
        "入線着順",
        "血統登録番号",
        "騎手コード",
        "調教師コード",
        "毛色",
        "生年月日",
    ],
    inplace=True,
)

In [24]:
df.columns

Index(['年', '月', '日', '回次', '場所', '日次', 'レース番号', 'レース名', 'クラスコード', '芝・ダ',
       'トラックコード', '距離', '馬場状態', '馬名', '性別', '年齢', '騎手名', '斤量', '頭数', '馬番',
       '確定着順', '異常コード', '着差タイム', '人気順', '走破タイム', '走破時計', '補正タイム', '通過順1',
       '通過順2', '通過順3', '通過順4', '上がり3Fタイム', '馬体重', '調教師', '所属地', '賞金', 'レースID',
       '馬主名', '生産者名', '父馬名', '母馬名', '母の父馬名', '単勝オッズ', 'PCI', 'レース日'],
      dtype='object')

In [25]:
df.dtypes

年                    int64
月                    int64
日                    int64
回次                   int64
場所                  object
日次                   int64
レース番号                int64
レース名                object
クラスコード               int64
芝・ダ                 object
トラックコード              int64
距離                   int64
馬場状態                object
馬名                  object
性別                  object
年齢                   int64
騎手名                 object
斤量                 float64
頭数                   int64
馬番                   int64
確定着順                 int64
異常コード                int64
着差タイム              float64
人気順                  int64
走破タイム              float64
走破時計                object
補正タイム                int64
通過順1                 int64
通過順2                 int64
通過順3                 int64
通過順4                 int64
上がり3Fタイム           float64
馬体重                  int64
調教師                 object
所属地                 object
賞金                   int64
レースID               object
馬

In [27]:
cond = df["異常コード"] == 0
df[cond].sort_values("通過順4")[
    ["場所", "距離", "通過順1", "通過順2", "通過順3", "通過順4"]
    ]

,場所,距離,通過順1,通過順2,通過順3,通過順4
348225,阪神,2400,1,1,1,1
367085,小倉,1800,5,4,1,1
83661,東京,1600,0,0,1,1
367061,小倉,1200,0,0,1,1
367046,小倉,1700,1,1,1,1
...,...,...,...,...,...,...
123194,中京,2200,17,17,18,18
301578,京都,1400,0,0,18,18
167893,新潟,1200,0,0,18,18
57339,阪神,1400,0,0,18,18


In [28]:
# =======================================
#              トラックコード
# =======================================
# 芝 = 0
# ダート = 1
# 障害・直線・芝 = 2
# 障害・直線・ダート = 3
# 芝・外回り = 8
df["トラックコード"].unique()

array([1, 0, 8, 3, 2])

In [29]:
# https://targetfaq.jra-van.jp/faq/detail?site=SVKNEGBV&id=706
# =======================================
#              クラスコード
# =======================================
# 7　　　　  未勝利
# 11	　　未出走
# 15	　　新馬
# 19	　　400万下
# 23	　　500万下
# 39	　　900万下
# 43	　　1000万下
# 63	　　1500万下
# 67	　　1600万下
# 　　1600万下
# 131	　　重賞以外のオープン
# 147	　　グレード無し重賞
# 163	　　G3
# 179	　　G2
# 195	　　G1
# =======================================

# df["クラスコード"].unique()
df["クラスコード"].unique()

array([  7,  23,  67, 163,  43, 179, 131, 115, 195,  15, 147])

In [30]:
# ====================================
#              レースID
# ====================================
#
#            ppyyknrruu
#
# pp:場所コード
# yy:西暦下2桁
# k・kk:回次
# n・nn:日次
# rr:レース番号
# uu:馬番
#
# ※ 場所コード先頭に0がつくケースとつかないケースあり

# ====================================
#              場所コード
# ====================================
# 01: 札幌
# 02: 函館
# 03: 福島
# 04: 新潟
# 05: 東京
# 06: 中山
# 07: 中京
# 08: 京都
# 09: 阪神
# 10: 小倉

df[["レースID", "補正タイム", "走破タイム", "上がり3Fタイム"]].sample(3)

,レースID,補正タイム,走破タイム,上がり3Fタイム
293724,0922270512,66,124.5,35.7
359123,919250315,68,116.9,40.3
24758,619450115,71,74.1,38.3


In [32]:
# 距離ローテ
df = df.sort_values(["馬名", "レース日"])

df["前走距離"] = (
    df
    # ← PARTITION BY 馬名
    .groupby("馬名")["距離"]
    # ← LAG(distance, 1)
    .shift(1)
)
df["距離延長"] = (df["距離"] >= df["前走距離"] + 200).fillna(False)
df["距離短縮"] = (df["距離"] <= df["前走距離"] - 200).fillna(False)

In [33]:
# 芝・ダートのローテ
df["前走芝・ダ"] = (
    df
    # PARTITION BY 馬名
    .groupby("馬名")["芝・ダ"]
    # LAG(芝・ダ, 1)
    .shift(1)
)

df["芝→ダ"] = np.where(
    df["前走芝・ダ"].notna()
    & (df["芝・ダ"] != df["前走芝・ダ"])
    & (df["前走芝・ダ"] == "芝"),
    True,  # 前走と違う ⇒ 変更あり
    False,  # 前走と同じ or 前走なし
)
df["ダ→芝"] = np.where(
    df["前走芝・ダ"].notna()
    & (df["芝・ダ"] != df["前走芝・ダ"])
    & (df["前走芝・ダ"] == "ダ"),
    True,  # 前走と違う ⇒ 変更あり
    False,  # 前走と同じ or 前走なし
)

In [34]:
# 騎手のローテ
df["前走騎手名"] = df.groupby("馬名")["騎手名"].shift(1)

df["乗り替わり"] = np.where(
    df["前走騎手名"].notna() & (df["騎手名"] != df["前走騎手名"]), True, False
)

# 判定したい騎手名をリストで用意
top_jockeys = ["ルメール", "レーン", "モレイラ", "Ｃ．デム"]

# True / False のブール Series を作成
is_top_jockey = df["前走騎手名"].isin(top_jockeys)
is_not_top_jockey = ~df["騎手名"].isin(top_jockeys)


df["鞍上弱化"] = np.where(
    df["前走騎手名"].notna()
    & (df["騎手名"] != df["前走騎手名"])
    & is_top_jockey
    & is_not_top_jockey,
    True,
    False,
)

In [35]:
# 前走通過順
df["前走通過順4"] = (
    df
    # PARTITION BY 馬名
    .groupby("馬名")["通過順4"]
    # LAG(芝・ダ, 1)
    .shift(1)
)

In [38]:
df.loc[:, "レースID"] = df["レースID"].astype(str)
mask = df["レースID"].str.len() == 9
# 先頭に "0" を付けて10桁に
df.loc[mask, "レースID"] = df.loc[mask, "レースID"].str.zfill(10)
# 馬番を削り8桁に
df.loc[:, "レースID"] = df["レースID"].str[:-2]

In [53]:
df_right = time_df()
df_right["レースID"] = df_right["レースID"].astype(str)
mask = df_right["レースID"].str.len() == 7
# 先頭に "0" を付けて8桁に
df_right.loc[mask, "レースID"] = df_right.loc[mask, "レースID"].str.zfill(8)
#df_right.drop(columns=["基準タイム", "開催名"], inplace=True)

In [41]:
df_merged = pd.merge(df, df_right, on='レースID', how='left')

In [42]:
df_merged

,年,月,日,回次,場所,日次,レース番号,レース名,クラスコード,芝・ダ,...,距離延長,距離短縮,前走芝・ダ,芝→ダ,ダ→芝,前走騎手名,乗り替わり,鞍上弱化,前走通過順4,基準タイム(秒)
0,23,10,22,4,東京,7,4,新馬,15,ダ,...,False,False,NaN,False,False,NaN,False,False,NaN,87.2
1,24,2,11,2,小倉,2,1,未勝利,7,ダ,...,True,False,ダ,False,False,菅原明良,True,False,12.0,106.4
2,15,1,17,1,中京,1,8,500万下・若,23,ダ,...,False,False,NaN,False,False,NaN,False,False,NaN,115.2
3,15,1,24,1,中京,3,9,500万下,23,ダ,...,False,False,ダ,False,False,長岡禎仁,False,False,2.0,114.4
4,15,3,14,2,中京,1,6,500万下,23,ダ,...,False,False,ダ,False,False,長岡禎仁,True,False,5.0,121.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
504662,24,10,5,5,京都,1,9,りんどう･1勝,23,芝,...,True,False,芝,False,False,浜中俊,True,False,2.0,NaN
504663,24,11,2,6,京都,1,11,ファンタG3,163,芝,...,False,False,芝,False,False,シュタル,False,False,6.0,NaN
504664,25,2,10,1,京都,3,10,エルフィ(L),115,芝,...,True,False,芝,False,False,シュタル,True,False,8.0,NaN
504665,25,4,13,2,阪神,6,11,桜花賞G1,195,芝,...,False,False,芝,False,False,浜中俊,False,False,2.0,NaN


In [43]:
df_merged[["基準タイム(秒)", "レースID", "走破タイム"]].sort_values("レースID")

,基準タイム(秒),レースID,走破タイム
191212,89.6,01151101,90.8
395777,89.6,01151101,89.5
302830,89.6,01151101,91.7
133428,89.6,01151101,90.9
469662,89.6,01151101,90.3
...,...,...,...
208339,NaN,10251C12,106.5
133211,NaN,10251C12,105.4
395249,NaN,10251C12,107.5
290795,NaN,10251C12,106.1


In [52]:
cond = df_merged["基準タイム(秒)"].isna()
df_merged[cond][['場所','年','月','日','回次','日次','レース日',"基準タイム(秒)", "レースID", "走破タイム"]].sort_values("レースID")

,場所,年,月,日,回次,日次,レース日,基準タイム(秒),レースID,走破タイム
386082,札幌,24,7,20,1,1,2024-07-20,NaN,01241101,68.8
492051,札幌,24,7,20,1,1,2024-07-20,NaN,01241101,70.1
470810,札幌,24,7,20,1,1,2024-07-20,NaN,01241101,70.0
492122,札幌,24,7,20,1,1,2024-07-20,NaN,01241101,69.4
296976,札幌,24,7,20,1,1,2024-07-20,NaN,01241101,69.1
...,...,...,...,...,...,...,...,...,...,...
140885,小倉,25,3,2,1,12,2025-03-02,NaN,10251C12,106.7
133211,小倉,25,3,2,1,12,2025-03-02,NaN,10251C12,105.4
89288,小倉,25,3,2,1,12,2025-03-02,NaN,10251C12,105.2
208339,小倉,25,3,2,1,12,2025-03-02,NaN,10251C12,106.5


In [81]:
df_right.sort_values('レースID')

,レースID,基準タイム,開催名,基準タイム(秒)
155,01151101,1.29.6,2015年1回札幌1日目 1R,89.6
156,01151102,1.00.5,2015年1回札幌1日目 2R,60.5
157,01151103,2.34.8,2015年1回札幌1日目 3R,154.8
158,01151104,1.47.0,2015年1回札幌1日目 4R,107.0
159,01151105,1.30.5,2015年1回札幌1日目 5R,90.5
...,...,...,...,...
29456,10242808,1.46.4,2024年2回小倉8日目 8R,106.4
29457,10242809,2.42.3,2024年2回小倉8日目 9R,162.3
29458,10242810,1.08.9,2024年2回小倉8日目10R,68.9
29459,10242811,2.01.8,2024年2回小倉8日目11R,121.8


In [83]:
cond = (df_right['開催名'].str.contains('2024年1回札幌'))
df_right[cond]

,レースID,基準タイム,開催名,基準タイム(秒)


In [51]:
len(df['レースID'].unique())

36010

In [86]:
tmp = pd.read_csv('../csv/基準タイム/2004-2024/041京.CSV',encoding='cp932', header=None)
